In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc

In [2]:
%matplotlib inline

In [3]:
# Configure data path
data_path = 'dataset/'

### Overview of the approach

Since NicheNet only filtered sets of expressed genes as ligands for the "sender cluster" and receptors for the "receiver cluster", and another set of marker genes / gene sets of interest as the affected downstream target genes. After the pre-selectio process, the inferred Ligand-Receptor scores only depend on the prior network values without considering the expresion counts in specific datasets, our task is to update such prior score

Assume we are interested in the active Ligand-Receptor interactions between sender cluster $A$ and receiver cluster $B$. <br>
Denote $G_l^A$ as the set of ligands expressed in cluster $A$, $G_r^B$ as the set of receptors expressed in cluster $B$. <br>
For all $g_i^A \in G_l^A$, $g_j^B \in G_r^B$ (the count values across all cells for gene $i$ and $j$ in two clusters), we define the weight representing relative expression activities:

$$w_{i,j}^{expr} = \dfrac{\bar{g}_i^A}{\dfrac{1}{|G_l^A|}\sum_k \bar{g}_k^A} \cdot \dfrac{\bar{g}_j^B}{\dfrac{1}{|G_r^B|}\sum_k \bar{g}_k^B}$$

In short, our assumptions are that, the ligand-receptor interaction potential can be updated by multipling the NicheNet prior networks and the expression weights definede here, and that the higher expression values a ligand-receptor prior have, the higher their interaction potential should have. Therefore, the final ligand-receptor prior interaction potential score is updated via: <br><br>
$$ w_{i,j}^{new} = \alpha \cdot w_{i,j}^{prior} + (1-\alpha)\cdot w_{i,j}^{prior}\cdot w_{i,j}^{expr}$$

We realize that the original prior potential score in the ligand-receptor networks is an integrated network coming from large database and multiple sources. Therefore, we bias the update scores towards the original values by assigning the $\alpha$ parameter. In our attempt, we set $\alpha$ to be 0.8.

### Load datasets

Load Ligand Receptor prior network

In [4]:
lr_sig_weights = pd.read_csv(data_path + 'lr_sig.csv', index_col=[0])
lr_sig_weights.head()

/home/yinuo/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,from,to,weight
1,A1BG,ABCC6,0.421644
2,A1BG,ACE2,0.100741
3,A1BG,ADAM10,0.096990
4,A1BG,AGO1,0.052459
5,A1BG,AKT1,0.085535


Load expressed genes

In [7]:
expressed_genes = pd.read_csv(data_path + 'expressed_genes.csv')
expressed_genes.head()

,Tumor,Macrophage 1,Macrophage 2
0,ACTB,LAPTM5,RPL17
1,GAPDH,METTL21A,HCLS1
2,MTRNR2L10,B2M,EIF4G2
3,MTRNR2L2,PABPC1,AKAP13
4,MTRNR2L8,LOC100131257,UGDH-AS1


Load preprocessed, normalized & log-transformed count matrix

In [8]:
counts = pd.read_csv(data_path + 'preprocessed_counts.csv', index_col=[0,1], header=[0])
counts.head()

,,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,AAAS,AACS,AACSP1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
Cell_ID,Cluster_ID,,,,,,,,,,,,,,,,,,,,,
MGH42_P7_A01,Macrophage 2,1.1928,0.0,0.000000,7.04390,0.0000,0.00000,0.0,0.0000,0.15316,0.0,...,0.00000,0.00000,0.0,0.090853,0.23879,0.00000,0.27143,2.7058,0.7137,0.0000
MGH42_P7_A02,Macrophage 2,0.0000,0.0,0.094912,7.60950,0.0000,0.00000,0.0,0.0000,0.96421,0.0,...,0.26183,0.00000,0.0,0.000000,0.37518,0.37072,0.44467,2.1077,1.1434,0.0000
MGH42_P7_A03,Tumor 5,0.0000,0.0,0.000000,0.77062,0.0000,0.19535,0.0,2.4662,2.99400,0.0,...,0.42975,3.14010,0.0,2.068300,0.76043,0.47508,3.13260,0.0000,0.0000,3.3404
MGH42_P7_A04,Macrophage 2,0.0000,0.0,0.000000,7.61460,0.0000,0.32077,0.0,0.0000,0.00000,0.0,...,0.00000,0.58688,0.0,0.000000,0.78157,0.15056,1.48650,3.8459,0.0000,0.0000
MGH42_P7_A05,Tumor 5,0.0000,0.0,0.000000,0.00000,2.0339,0.20664,0.0,1.9800,5.13340,0.0,...,0.00000,0.00000,0.0,0.000000,2.60240,0.47508,4.03830,3.8272,1.5514,4.8840


Separate count matrix to tumor, macrophage 1 & macrophage 2

In [8]:
tumor_labels = ['Tumor 1', 'Tumor 2', 'Tumor 3', 'Tumor 4', 'Tumor 5', 'Tumor 6', 'Tumor 7', 'Tumor 8', 'Tumor 9', 'Tumor 10', 'Tumor 11']
tumor_counts = counts.loc[counts.index.get_level_values('Cluster_ID').isin(tumor_labels)]
tumor_counts = tumor_counts[expressed_genes['Tumor']] # subsetting expressed genes
tumor_counts.head()

,,ACTB,GAPDH,MTRNR2L10,MTRNR2L2,MTRNR2L8,MTRNR2L1,EEF1A1,KCNQ1OT1,RPL15,RPL41,...,CENPB,PPP2R3B,FAM109A,CDK8,DDX26B,MC1R,NOX4,LOC644669,CCM2,C12orf45
Cell_ID,Cluster_ID,,,,,,,,,,,,,,,,,,,,,
MGH42_P7_A03,Tumor 5,7.0265,9.7547,7.8646,13.095,11.363,8.8390,8.4922,4.5502,5.8136,7.4527,...,0.00000,1.2467,0.0000,0.0,2.7461,0.00000,0.00000,0.82212,2.47920,0.0
MGH42_P7_A05,Tumor 5,8.9033,9.0750,6.8073,11.841,10.185,7.7003,7.9844,3.5463,6.0516,6.2393,...,0.00000,0.0000,0.0000,0.0,0.0000,0.00000,0.00000,0.00000,0.00000,0.0
MGH42_P7_A07,Tumor 5,8.2244,9.3207,7.7504,12.771,11.218,8.6307,7.7845,3.2084,7.1153,7.4233,...,1.74420,0.0000,0.9782,0.0,0.0000,0.00000,0.14796,0.31846,0.00000,0.0
MGH42_P7_A09,Tumor 5,6.2788,8.0434,9.2514,14.288,12.581,10.0490,8.4716,2.9128,6.7977,6.8614,...,0.00000,0.0000,0.0000,0.0,3.2362,0.39835,0.00000,0.56169,0.00000,0.0
MGH42_P7_B03,Tumor 5,6.4840,8.3221,8.7567,13.561,12.091,9.4485,8.3216,2.9978,6.7263,8.2723,...,0.81065,0.0000,0.0000,0.0,0.0000,1.61780,0.16993,0.00000,0.70752,0.0


In [9]:
macro1_counts = counts.loc[counts.index.get_level_values('Cluster_ID') == 'Macrophage 1']
macro1_counts = macro1_counts[expressed_genes['Macrophage 1']] # subsetting expressed genes
macro1_counts.head()

,,LAPTM5,METTL21A,B2M,PABPC1,LOC100131257,RPL15,HLA-E,RPL10,SHISA9,MTRNR2L8,...,GALK2,CDR2L,LOC100505687,TADA2A,CUEDC1,EFCAB13,CDK5,NELFB,MED25,FOXN3-AS2
Cell_ID,Cluster_ID,,,,,,,,,,,,,,,,,,,,,
MGH42_P8_E03,Macrophage 1,8.9693,3.3748,9.1338,6.7706,1.42810,7.6370,8.6759,8.1628,3.52530,10.2010,...,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.0000,0.0,0.0,0.0
MGH42_P3_G01,Macrophage 1,7.3885,2.3383,11.2150,6.9208,0.36177,6.5076,7.6082,7.6511,0.62106,10.3750,...,0.0,0.22527,0.00000,0.0,0.00000,0.0,3.8572,0.0,0.0,0.0
MGH42_P3_H07,Macrophage 1,6.7197,1.9797,9.0486,4.9750,0.23020,6.2133,6.3972,7.8733,0.17121,8.9291,...,0.0,0.00000,0.00000,0.0,0.27023,0.0,0.0000,0.0,0.0,0.0
MGH43_P3_A02,Macrophage 1,7.8964,3.6232,9.3455,6.3007,1.37130,6.7854,7.9643,7.2624,3.03860,11.0530,...,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.0000,0.0,0.0,0.0
MGH43_P3_A03,Macrophage 1,6.7911,4.0064,9.7708,7.6374,1.08200,7.3301,6.7929,7.8536,2.49160,10.8000,...,0.0,0.00000,0.43189,0.0,0.00000,0.0,0.0000,0.0,0.0,0.0


In [10]:
macro2_counts = counts.loc[counts.index.get_level_values('Cluster_ID') == 'Macrophage 2']
macro2_counts = macro2_counts[expressed_genes['Macrophage 2']] # subsetting expressed genes
macro2_counts.head()

,,RPL17,HCLS1,EIF4G2,AKAP13,UGDH-AS1,SAT1,UCP2,EIF1,KLF6,EEF2,...,ABCA5,SEL1L3,TAB3,EFHA2,RPIA,ULK4P1,TFB2M,SPSB2,TRMT61B,SYNJ2BP-COX16
Cell_ID,Cluster_ID,,,,,,,,,,,,,,,,,,,,,
MGH42_P7_A01,Macrophage 2,8.5657,6.7679,3.7504,4.4772,5.4393,11.1010,5.0669,7.3556,4.0995,6.0154,...,0.0,0.000000,0.00000,0.0,3.1419,2.3663,0.0,0.0000,0.0,0.00000
MGH42_P7_A02,Macrophage 2,7.3692,5.4984,5.9190,3.7862,6.5638,8.8500,4.8784,7.3655,6.0553,5.8143,...,0.0,0.000000,0.00000,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.00000
MGH42_P7_A04,Macrophage 2,7.0010,5.9668,5.7454,4.2272,5.9017,10.4810,6.2554,7.1527,5.8621,5.5808,...,0.0,0.000000,0.00000,0.0,0.0000,0.0000,0.0,2.1754,0.0,0.00000
MGH42_P7_A11,Macrophage 2,7.3043,6.7836,4.7256,4.9411,5.4910,10.8450,3.2403,7.4554,6.4375,6.0817,...,0.0,0.096262,0.24489,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.39835
MGH42_P7_A12,Macrophage 2,6.9131,6.6663,3.5712,3.6567,5.2491,9.3517,6.2907,8.0092,6.5853,2.5408,...,0.0,0.000000,0.00000,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.00000


### Update network weights specific to each cluster interaction

In [120]:
def update_lr_weight(sender_counts, receiver_counts, lr_sig_weights, alpha=0.8):
    lr_network = lr_sig_weights.copy() # update weights on the copy of the original lr-network
    ligands = np.intersect1d(sender_counts.columns, lr_network['from'])
    ligand_set = set(ligands)
    receptors = np.intersect1d(receiver_counts.columns, lr_network['to'])
    receptor_set = set(receptors)
    
    # update lr network with only expressedd ligand & receptors from the clusters
    mask_ligands = lr_network['from'].apply(lambda x: x in ligand_set)
    mask_receptors = lr_network['to'].apply(lambda x: x in receptor_set)
    lr_network = lr_network[np.bitwise_and(mask_ligands, mask_receptors)]
    lr_network.reset_index(inplace=True, drop=True)

    ligand_counts = sender_counts[ligands]
    receptor_counts = receiver_counts[receptors]
    ligand_weights = ligand_counts.mean(0) / ligand_counts.mean().mean()
    receptor_weights = receptor_counts.mean(0) / receptor_counts.mean().mean()
    
    # update weights
    weight_expr = lr_network.apply(lambda x: ligand_weights[x['from']] * receptor_weights[x['to']], axis=1)
    lr_network['weight'] = alpha * lr_network['weight'] + (1-alpha) * weight_expr
    
    return lr_network

    

In [177]:
# tumor --> macrophage 1
lr_network_t_m1 = update_lr_weight(tumor_counts, macro1_counts, lr_sig_weights)

# tumor --> macrophage 2
lr_network_t_m2 = update_lr_weight(tumor_counts, macro2_counts, lr_sig_weights)

# macrophage 1 --> tumor
lr_network_m1_t = update_lr_weight(macro1_counts, tumor_counts, lr_sig_weights)

# macrophage 2 --> tumor
lr_network_m2_t = update_lr_weight(macro2_counts, tumor_counts, lr_sig_weights)

Save updated lr-network weights:

In [184]:
lr_network_t_m1.to_csv(data_path + 'lr_sig_tumor_macro1.csv', index=False)
lr_network_t_m2.to_csv(data_path + 'lr_sig_tumor_macro2.csv', index=False)
lr_network_m1_t.to_csv(data_path + 'lr_sig_macro1_tumor.csv', index=False)
lr_network_m2_t.to_csv(data_path + 'lr_sig_macro2_tumor.csv', index=False)